# 🧪 ADK Application Testing

This notebook demonstrates how to test an ADK (Agent Development Kit) application.
It covers both local and remote testing, both with Agent Engine and Cloud Run.

> **Note**: This notebook assumes that the agent files are stored in the `app` folder. If your agent files are located in a different directory, please update all relevant file paths and references accordingly.

## Install dependencies

In [1]:
!pip install google-cloud-aiplatform --upgrade

### Import libraries

In [2]:
import json

import requests
import vertexai

### Initialize Vertex AI Client

In [3]:
# Initialize the Vertex AI client
LOCATION = "us-central1"

client = vertexai.Client(
    location=LOCATION,
)

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


## If you are using Agent Engine
See more documentation at [Agent Engine Overview](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/overview)

### Remote Testing

In [6]:
# Set to None to auto-detect from ./deployment_metadata.json, or specify manually
# "projects/PROJECT_ID/locations/us-central1/reasoningEngines/ENGINE_ID"
REASONING_ENGINE_ID = None

if REASONING_ENGINE_ID is None:
    try:
        with open("../deployment_metadata.json") as f:
            metadata = json.load(f)
            REASONING_ENGINE_ID = metadata.get("remote_agent_engine_id")
    except (FileNotFoundError, json.JSONDecodeError):
        pass

print(f"Using REASONING_ENGINE_ID: {REASONING_ENGINE_ID}")
# Get the existing agent engine
remote_agent_engine = client.agent_engines.get(name=REASONING_ENGINE_ID)

Using REASONING_ENGINE_ID: None


ClientError: 404 Not Found. {'message': '<!DOCTYPE html>\n<html lang=en>\n  <meta charset=utf-8>\n  <meta name=viewport content="initial-scale=1, minimum-scale=1, width=device-width">\n  <title>Error 404 (Not Found)!!1</title>\n  <style>\n    *{margin:0;padding:0}html,code{font:15px/22px arial,sans-serif}html{background:#fff;color:#222;padding:15px}body{margin:7% auto 0;max-width:390px;min-height:180px;padding:30px 0 15px}* > body{background:url(//www.google.com/images/errors/robot.png) 100% 5px no-repeat;padding-right:205px}p{margin:11px 0 22px;overflow:hidden}ins{color:#777;text-decoration:none}a img{border:0}@media screen and (max-width:772px){body{background:none;margin-top:0;max-width:none;padding-right:0}}#logo{background:url(//www.google.com/images/branding/googlelogo/1x/googlelogo_color_150x54dp.png) no-repeat;margin-left:-5px}@media only screen and (min-resolution:192dpi){#logo{background:url(//www.google.com/images/branding/googlelogo/2x/googlelogo_color_150x54dp.png) no-repe

In [ ]:
async for event in remote_agent_engine.async_stream_query(
    message="hi!", user_id="test"
):
    print(event)

In [ ]:
remote_agent_engine.register_feedback(
    feedback={
        "score": 5,
        "text": "Great response!",
        "invocation_id": "test-invocation-123",
        "user_id": "test",
    }
)

### Local Testing

You can import directly the AgentEngineApp class within your environment. 
To run the agent locally, follow these steps:
1. Make sure all required packages are installed in your environment
2. The recommended approach is to use the same virtual environment created by the 'uv' tool
3. You can set up this environment by running 'make install' from your agent's root directory
4. Then select this kernel (.venv folder in your project) in your Jupyter notebook to ensure all dependencies are available

In [ ]:
# Uncomment the following lines if you're not using the virtual environment created by uv
# import sys
# sys.path.append("../")

In [ ]:
from app.agent import root_agent
from app.agent_engine_app import AgentEngineApp

agent_engine = AgentEngineApp(agent=root_agent)

In [ ]:
async for event in agent_engine.async_stream_query(message="hi!", user_id="test"):
    print(event)

## If you are using Cloud Run

#### Remote Testing

For more information about authenticating HTTPS requests to Cloud Run services, see:
[Cloud Run Authentication Documentation](https://cloud.google.com/run/docs/triggering/https-request)

Remote testing involves using a deployed service URL instead of localhost.

Authentication is handled using GCP identity tokens instead of local credentials.

In [7]:
ID_TOKEN = get_ipython().getoutput("gcloud auth print-identity-token -q")[0]

In [8]:
SERVICE_URL = "https://adk-agent-cloudrun-demo-913703049174.us-central1.run.app"  # Replace with your Cloud Run service URL

You'll need to first create a Session

In [9]:
user_id = "test_user_123"
session_data = {"state": {"preferred_language": "English", "visit_count": 1}}

session_url = f"{SERVICE_URL}/apps/app/users/{user_id}/sessions"
headers = {"Content-Type": "application/json", "Authorization": f"Bearer {ID_TOKEN}"}

session_response = requests.post(session_url, headers=headers, json=session_data)
print(f"Session creation status code: {session_response.status_code}")
print(f"Session creation response: {session_response.json()}")
session_id = session_response.json()["id"]

Session creation status code: 200
Session creation response: {'id': '95cb141a-2b34-4a4b-be03-0c952b59e56b', 'appName': 'app', 'userId': 'test_user_123', 'state': {'preferred_language': 'English', 'visit_count': 1}, 'events': [], 'lastUpdateTime': 1762263214.9052832}


Then you will be able to send a message

In [10]:
message_data = {
    "app_name": "app",
    "user_id": user_id,
    "session_id": session_id,
    "new_message": {"role": "user", "parts": [{"text": "Hello! Weather in New york?"}]},
    "streaming": True,
}

message_url = f"{SERVICE_URL}/run_sse"
message_response = requests.post(
    message_url, headers=headers, json=message_data, stream=True
)

print(f"Message send status code: {message_response.status_code}")

# Print streamed response
for line in message_response.iter_lines():
    if line:
        line_str = line.decode("utf-8")
        if line_str.startswith("data: "):
            event_json = line_str[6:]
            event = json.loads(event_json)
            print(f"Received event: {event}")

Message send status code: 200
Received event: {'content': {'parts': [{'thoughtSignature': 'CiwB4_H_Xjsr9UVuwswN-_FRcg1feIDreX8LdVyZFY0qftTevgvhWNYXEq_EagplAePx_17To9iqmVY0luRXulrnD_jfkEozRttSKJLcGvWc1dkeUJ4L5HkyounAwlzUo0XVrLl8KAC-eksPk4E-i7FtTwDuhZ6Avftu9cMLVIRCOrY33fLnE0mQTZy563LhCfShIfgKhgEB4_H_XmqHxzHdDUloUHeERau9oGAGUjW1m4PoYidYOzAVpwGufDIpYwtTnlNc3FVv0g0rMsRENWBzr86ZGMLhR5j2dH-qPXeBW2Il24ysApQnrMEgBGjEGdN8yEjW3L3uz3ru7N2d7tZDkYAWynzFTkd6cgqCw4H80Rtr_J_kaTN-GNSLJgo4AePx_156Crlw8oNpm8bSySkr7smluCcC_XJIeMt66HV-yKJjusR3zGPmO7XzcEvnZGbJS-ZmLCI=', 'functionCall': {'id': 'adk-41536a13-5ce9-4353-949f-521f2a9b8ef4', 'args': {'query': 'New York'}, 'name': 'get_weather'}}], 'role': 'model'}, 'finishReason': 'STOP', 'usageMetadata': {'candidatesTokenCount': 6, 'candidatesTokensDetails': [{'modality': 'TEXT', 'tokenCount': 6}], 'promptTokenCount': 152, 'promptTokensDetails': [{'modality': 'TEXT', 'tokenCount': 152}], 'thoughtsTokenCount': 51, 'totalTokenCount': 209, 'trafficType': 'ON_DEMAND'

### Local Testing

> You can run the application locally via the `make local-backend` command.

#### Create a session
 Create a new session with user preferences and state information


In [ ]:
user_id = "test_user_123"
session_data = {"state": {"preferred_language": "English", "visit_count": 1}}

session_url = f"http://127.0.0.1:8000/apps/app/users/{user_id}/sessions"
headers = {"Content-Type": "application/json"}

session_response = requests.post(session_url, headers=headers, json=session_data)
print(f"Session creation status code: {session_response.status_code}")
print(f"Session creation response: {session_response.json()}")
session_id = session_response.json()["id"]

#### Send a message
Send a message to the backend service and receive a streaming response


In [ ]:
message_data = {
    "app_name": "app",
    "user_id": user_id,
    "session_id": session_id,
    "new_message": {"role": "user", "parts": [{"text": "Hello! Weather in New york?"}]},
    "streaming": True,
}

message_url = "http://127.0.0.1:8000/run_sse"
message_response = requests.post(
    message_url, headers=headers, json=message_data, stream=True
)

print(f"Message send status code: {message_response.status_code}")

# Print streamed response
for line in message_response.iter_lines():
    if line:
        line_str = line.decode("utf-8")
        if line_str.startswith("data: "):
            event_json = line_str[6:]
            event = json.loads(event_json)
            print(f"Received event: {event}")